In [1]:
import torch
from torch import nn
import torch.optim as optim
import torch.nn.functional as F

# 1. Dataset setup
# Define a small set of users, items, and ratings.
num_users = 4
num_items = 6

# Known ratings in the form (user_id, item_id, rating)
ratings = [
    (0, 0, 5.0),
    (0, 1, 3.0),
    (0, 2, 1.0),
    (1, 0, 4.0),
    (1, 3, 2.0),
    (2, 1, 4.0),
    (2, 2, 5.0),
    (2, 4, 2.0),
    (3, 3, 5.0),
    (3, 4, 3.0)
]

# Convert lists of users, items, and ratings to tensors
users = torch.tensor([r[0] for r in ratings], dtype=torch.long)
items = torch.tensor([r[1] for r in ratings], dtype=torch.long)
ratings_values = torch.tensor([r[2] for r in ratings], dtype=torch.float)

# Define content features for each item (e.g., binary genre tags)
# Here we assume 3 genres; each item has a 3D binary feature vector.
item_features = {
    0: [1, 0, 0],  # Item 0: genre vector [1,0,0]
    1: [1, 1, 0],  # Item 1: genres 0 and 1
    2: [0, 1, 0],  # Item 2: genre 1
    3: [0, 0, 1],  # Item 3: genre 2
    4: [0, 1, 1],  # Item 4: genres 1 and 2
    5: [1, 0, 1]   # Item 5: genres 0 and 2
}

# Create a tensor of item features corresponding to each rating entry
features_list = [item_features[item_id] for item_id in items.tolist()]
features = torch.tensor(features_list, dtype=torch.float)

# Define a PyTorch Dataset to feed the data in batches
class RatingsDataset(torch.utils.data.Dataset):
    def __init__(self, users, items, features, ratings):
        self.users = users
        self.items = items
        self.features = features
        self.ratings = ratings
        
    def __len__(self):
        return len(self.ratings)
    
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.features[idx], self.ratings[idx]

dataset = RatingsDataset(users, items, features, ratings_values)
loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True)

# 2. Model definition
class HybridRecommender(nn.Module):
    def __init__(self, num_users, num_items, content_dim, emb_dim=4, hidden_dim=8):
        super().__init__()
        # Embeddings for users and items (collaborative filtering part)
        self.user_emb = nn.Embedding(num_users, emb_dim)
        self.item_emb = nn.Embedding(num_items, emb_dim)
        # A linear layer to transform item content features (content-based part)
        self.content_fc = nn.Linear(content_dim, emb_dim)
        # Layers to combine all features
        self.fc1 = nn.Linear(emb_dim * 3, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)
        
    def forward(self, user, item, content):
        # Get user and item embeddings
        u = self.user_emb(user)          # shape: (batch_size, emb_dim)
        v = self.item_emb(item)          # shape: (batch_size, emb_dim)
        # Process content features
        c = F.relu(self.content_fc(content))  # shape: (batch_size, emb_dim)
        # Concatenate embeddings and content
        x = torch.cat([u, v, c], dim=1) # shape: (batch_size, emb_dim*3)
        x = F.relu(self.fc1(x))
        rating = self.fc2(x)
        return rating.squeeze()         # shape: (batch_size,)

# Instantiate the model
model = HybridRecommender(num_users, num_items, content_dim=3)

# Loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 3. Training loop
num_epochs = 100
for epoch in range(num_epochs):
    total_loss = 0.0
    for user_batch, item_batch, features_batch, rating_batch in loader:
        optimizer.zero_grad()
        # Predict ratings
        preds = model(user_batch, item_batch, features_batch)
        loss = criterion(preds, rating_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    # Print loss occasionally
    if epoch % 20 == 0 or epoch == num_epochs - 1:
        print(f"Epoch {epoch:3d}, Loss: {total_loss:.4f}")

# 4. Recommendation for a user
test_user = 0
# Find which items the user has already rated
rated_items = {item for (user, item, rating) in ratings if user == test_user}

# Predict ratings for all items not yet rated by the user
candidates = []
for item_id in range(num_items):
    if item_id not in rated_items:
        u = torch.tensor([test_user], dtype=torch.long)
        i = torch.tensor([item_id], dtype=torch.long)
        c = torch.tensor([item_features[item_id]], dtype=torch.float)
        pred_rating = model(u, i, c).item()
        candidates.append((item_id, pred_rating))

# Sort candidate items by predicted rating in descending order
candidates.sort(key=lambda x: x[1], reverse=True)

print(f"\nTop recommendations for user {test_user}:")
for item_id, score in candidates:
    print(f"Item {item_id} — predicted rating: {score:.2f}")


Epoch   0, Loss: 58.4519
Epoch  20, Loss: 3.8965
Epoch  40, Loss: 1.2053
Epoch  60, Loss: 0.7342
Epoch  80, Loss: 0.3058
Epoch  99, Loss: 0.0625

Top recommendations for user 0:
Item 5 — predicted rating: 2.28
Item 3 — predicted rating: 2.26
Item 4 — predicted rating: 1.26
